In [790]:
import Radar as R
import matplotlib.pyplot as plt
import numpy as np
import subprocess
import os
import time
import argparse
import pandas as pd
import time
from pyquaternion import Quaternion
from numpy import genfromtxt
import os
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import DBSCAN
from sklearn import metrics
import pandas as pd
from pyquaternion import Quaternion
from numpy.linalg import norm
import math
import importlib

In [791]:
importlib.reload(R)

<module 'Radar' from '/Users/shannonholmes/Desktop/Python Programs/AWN/Radar.py'>

In [792]:
frame_num = '1'
Frame_time = 0
## initialization ##
Cluster_dict = {}
Next_Cluster_dict = {}
Next_Velocity_dict = {}
## KalmanFilter for Client 1
KalmanMeasurements = np.zeros((4,1))
KalmanP = np.zeros((4,4,1))
Innovation = np.zeros((2,1))
KalmanF = np.zeros((4,2,1))
ConditionalX = np.zeros((4,1))
ConditionalP=np.identity(4)

## KalmanFilter for Client 2 

KalmanMeasurements2 = np.zeros((4,1))
KalmanP2 = np.zeros((4,4,1))
Innovation2 = np.zeros((2,1))
KalmanF2 = np.zeros((4,2,1))
ConditionalX2 = np.zeros((4,1))
ConditionalP2 = np.identity(4)

x = 0
client1 = R.client(0)
client2 = R.client(1)

Start_time = time.time()

In [793]:
data_df = pd.read_csv('/Users/shannonholmes/Desktop/Python Programs/AWN/test2.csv')
num_frames = data_df.iloc[:,1].nunique()
print("DEBUG: Number of unique frames: ", num_frames)

DEBUG: Number of unique frames:  50


In [794]:
data = R.getData(data_df)
Frame = R.rframe(frame_num,data,None,None,Start_time)

13140


In [795]:
labels,n_clusters_ = Frame.getCluster()

print(labels)
print(n_clusters_)

[0 0 0 ... 7 7 7]
14


In [796]:
CorePoints = np.zeros((n_clusters_,3))
CoreNum = np.zeros((n_clusters_))
CoreSum = np.zeros((n_clusters_,3))

for point in range(0,len(Frame.data)):
    if labels[point] != -1:
        CoreNum[labels[point]] = CoreNum[labels[point]] + 1
        CoreSum[labels[point]][0] = CoreSum[labels[point]][0] + Frame.data.loc[point].iat[3]
        CoreSum[labels[point]][1] = CoreSum[labels[point]][1] + Frame.data.loc[point].iat[4]
        CoreSum[labels[point]][2] = CoreSum[labels[point]][2] + Frame.data.loc[point].iat[5]

for corepoint_id in range(0,n_clusters_):
    CorePoints[corepoint_id] = CoreSum[corepoint_id]/CoreNum[corepoint_id]

KeyError: 0

In [ ]:
Cluster_dict = Next_Cluster_dict.copy()

Next_Cluster_dict = {}

if Cluster_dict:
    if len(Cluster_dict) >= n_clusters_:
        for corepoint_id in range (0,n_clusters_):
            Min_dis = 100
            Min_key = 100
            for keys in Cluster_dict:
                dis = (Cluster_dict[keys][0]-CorePoints[corepoint_id][0])**2+(Cluster_dict[keys][1]-CorePoints[corepoint_id][1])**2\
                        +(Cluster_dict[keys][2]-CorePoints[corepoint_id][2])**2
                if dis < Min_dis:
                    Min_key = keys
                    Min_dis = dis
            Next_Cluster_dict[Min_key] = CorePoints[corepoint_id]
    elif len(Cluster_dict) < n_clusters_:
        for keys in Cluster_dict:
            Min_dis = 100
            Min_core_id = 100
            for corepoint_id in range(0,n_clusters_):
                dis = (Cluster_dict[keys][0]-CorePoints[corepoint_id][0])**2+(Cluster_dict[keys][1]-CorePoints[corepoint_id][1])**2\
                        +(Cluster_dict[keys][2]-CorePoints[corepoint_id][2])**2
                if dis < Min_dis:
                    Min_core_id = corepoint_id
                    Min_dis = dis
            Next_Cluster_dict[keys] = CorePoints[Min_core_id]

    for keys in Cluster_dict:
        if keys in Next_Cluster_dict:
            Next_Velocity_dict[keys] = [(Next_Cluster_dict[keys][0] - Cluster_dict[keys][0]) / 0.18,
                                    (Next_Cluster_dict[keys][1] - Cluster_dict[keys][1]) / 0.18,
                                    (Next_Cluster_dict[keys][2] - Cluster_dict[keys][2]) / 0.18]
    
else:
    for corepoint_id in range(0,n_clusters_):
        Next_Cluster_dict[corepoint_id] = CorePoints[corepoint_id]

print(Next_Cluster_dict)

{0: array([ 0.60522164,  2.13589176, -0.9105395 ]), 1: array([ 1.57118226,  5.98173742, -0.45831769]), 2: array([ 0.47821276,  4.62249707, -0.88735821]), 3: array([0.58591127, 3.29079695, 1.81936381])}


In [ ]:
R.trackOrientation6D(Frame_time,client1)
R.trackOrientation6D(Frame_time,client2)
R.getVelocity(Frame_time,client1,client2)

In [ ]:
R.quaternion_to_euler(client1)
R.quaternion_to_euler(client2)
print(Next_Velocity_dict)

{}


In [ ]:
client1_id = Frame.findrouter(client1, Microframe, Next_Velocity_dict)
client2_id = Frame.findrouter(client2, Microframe, Next_Velocity_dict)

client1.id = 0
client2.id = 1

In [ ]:
KalmanMeasurements,KalmanP,Innovation,KalmanF,ConditionalX,ConditionalP = Frame.kalmanFilter\
             (client1,Next_Cluster_dict,KalmanMeasurements,KalmanP,Innovation,KalmanF,ConditionalX,ConditionalP)
        
KalmanMeasurements2,KalmanP2,Innovation2,KalmanF2,ConditionalX2,ConditionalP2 = Frame.kalmanFilter\
        (client2,Next_Cluster_dict,KalmanMeasurements2,KalmanP2,Innovation2,KalmanF2,ConditionalX2,ConditionalP2)

print(client1.x)
print(client2.x)

[0.48417731]
[1.25694581]


In [ ]:
Theta = Frame.getEstimate (client1,client2) 

In [ ]:
Beamangle = R.beamform_angle(Theta,client1)

Can beamform
Left side of center
